In [1]:
import pandas as pd
import numpy as np
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset
import warnings
warnings.filterwarnings("ignore")

# 1. Preprocessing

In [2]:
df_RtnI = pd.read_csv('./cache/LongShortLegRtns.csv', low_memory=False)
df_RtnJ = pd.read_csv('./cache/BalancedPanel.csv', low_memory=False)
rf = pd.read_csv('./data/F-F_Research_Data_Factors.csv').iloc[:,[0,4]]
rf.columns = ['yyyymm','rf']
df_RtnJ = df_RtnJ.merge(rf, on='yyyymm', how='left')
df_RtnJ['RET'] = df_RtnJ['RET'] - df_RtnJ['rf'] / 100
del df_RtnJ['rf']

In [3]:
df_RtnI

,yyyymm,MaxRet,RealizedVol,IdioVol3F,ReturnSkew3F,High52,ReturnSkew,IdioVolAHT,zerotrade1M,DolVol,...,DivInit,IndIPO,ExchSwitch,DivOmit,Spinoff,FirmAge,IndMom,betaVIX,VolSD,Mom6m
0,197001,-0.022823,-0.021712,-0.014996,-0.056396,0.071570,-0.065887,0.016471,-0.031486,-0.035241,...,0.017074,0.016909,0.024801,0.005470,0.013002,-0.059356,0.007943,-0.004913,-0.015119,0.114313
1,197002,-0.003664,-0.011480,-0.008748,-0.064380,0.134947,-0.072887,0.015311,-0.015795,-0.017999,...,-0.011912,-0.011691,0.037457,0.008600,-0.010427,0.003701,0.017070,-0.004446,-0.015417,0.113456
2,197003,0.029763,-0.002334,-0.005837,-0.074349,0.239440,-0.074835,0.008926,-0.026036,-0.018940,...,-0.060480,-0.059742,0.137510,0.057687,-0.047706,0.117005,0.049380,-0.017822,-0.035985,0.107710
3,197004,-0.007777,-0.025206,-0.028268,-0.077262,0.169261,-0.074618,-0.019653,-0.037155,-0.017177,...,-0.032768,-0.031981,0.054877,-0.031981,-0.025512,0.063894,0.057498,-0.003749,-0.044473,0.117711
4,197005,0.037362,0.022492,0.008892,-0.059927,0.170072,-0.049957,0.009913,-0.021427,-0.020260,...,-0.028564,-0.028162,-0.050876,-0.011446,-0.021607,0.060272,0.063794,-0.009540,-0.020458,0.148859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,202407,0.039772,0.036900,0.023779,0.017277,0.027705,0.018373,0.035582,0.032851,-0.017830,...,-0.003393,-0.002702,-0.026327,-0.085658,-0.002702,-0.001558,0.013956,0.052306,0.011174,0.013186
655,202408,0.032972,0.034728,0.029562,0.014829,0.027237,0.006424,0.041386,0.008148,-0.028561,...,-0.003679,-0.003341,-0.045473,-0.140709,-0.003341,-0.000956,0.000012,0.020603,0.006690,0.003788
656,202409,0.029980,0.030067,0.031553,0.008517,0.011594,0.005588,0.015008,0.007575,-0.007354,...,-0.020625,-0.020395,-0.020047,-0.184424,-0.020395,0.015681,0.034512,0.034884,0.007007,0.025307
657,202410,0.011822,0.015077,0.025644,0.019869,0.051094,0.003758,0.041436,0.021562,-0.019508,...,0.047765,0.047639,-0.017854,-0.118242,0.047639,0.016440,-0.001255,-0.019756,0.033127,0.028541


In [4]:
df_RtnJ

,permno,yyyymm,RET,MaxRet,RealizedVol,IdioVol3F,ReturnSkew3F,High52,ReturnSkew,IdioVolAHT,...,DivInit,IndIPO,ExchSwitch,DivOmit,Spinoff,FirmAge,IndMom,betaVIX,VolSD,Mom6m
0,10137.0,197001.0,0.117377,-0.035714,-0.019466,-0.016819,0.160126,0.762376,0.082765,-0.014904,...,0.0,0.0,0.0,0.0,0.0,-530.0,-0.030537,0.001931,-0.072657,-0.004711
1,10137.0,197002.0,0.014840,-0.036364,-0.014232,-0.013282,-0.190259,0.856436,-0.321733,-0.015210,...,0.0,0.0,0.0,0.0,0.0,-531.0,-0.059526,0.001931,-0.069922,-0.054476
2,10137.0,197003.0,-0.086160,-0.042945,-0.019876,-0.015761,1.236047,0.920635,0.763550,-0.015805,...,0.0,0.0,0.0,0.0,0.0,-532.0,0.062296,0.001931,-0.068552,0.046238
3,10137.0,197004.0,-0.073750,-0.017241,-0.009264,-0.008997,0.697642,0.855615,0.368652,-0.015908,...,0.0,0.0,0.0,0.0,0.0,-533.0,0.009572,0.001931,-0.068413,0.013934
4,10137.0,197005.0,-0.054696,-0.027027,-0.014745,-0.012134,-0.196592,0.796791,-0.226604,-0.016336,...,0.0,0.0,0.0,0.0,0.0,-534.0,-0.020445,0.001931,-0.069392,0.006259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608252,11267.0,202407.0,-0.053328,-0.043478,-0.022724,-0.016043,-0.822875,0.683908,0.045570,-0.017291,...,0.0,0.0,0.0,0.0,0.0,-441.0,0.175894,-0.000780,-1.677947,-0.119106
608253,11267.0,202408.0,0.054748,-0.043478,-0.022724,-0.016043,-0.822875,0.683908,0.045570,-0.017291,...,0.0,0.0,0.0,0.0,0.0,-441.0,0.175894,-0.000780,-1.677947,-0.119106
608254,11267.0,202409.0,0.304617,-0.043478,-0.022724,-0.016043,-0.822875,0.683908,0.045570,-0.017291,...,0.0,0.0,0.0,0.0,0.0,-441.0,0.175894,-0.000780,-1.677947,-0.119106
608255,11267.0,202410.0,-0.515385,-0.043478,-0.022724,-0.016043,-0.822875,0.683908,0.045570,-0.017291,...,0.0,0.0,0.0,0.0,0.0,-441.0,0.175894,-0.000780,-1.677947,-0.119106


In [5]:
df_RtnI.isna().any().any()

np.False_

In [6]:
df_RtnJ.isna().any().any()

np.False_

In [7]:
def standardize_cross_section(group):
    # 每一个除了 RET 之外的列都标准化处理
    for col in group.columns[3:]:
        if group[col].std() == 0:
            group[col] = 0
        else:
            group[col] = (group[col] - group[col].mean()) / group[col].std()
    return group

In [8]:
df_RtnJ = df_RtnJ.groupby('yyyymm').apply(standardize_cross_section).\
    reset_index(drop=True).sort_values(by=['yyyymm', 'permno']).reset_index(drop=True)

In [9]:
df_RtnJ

,permno,yyyymm,RET,MaxRet,RealizedVol,IdioVol3F,ReturnSkew3F,High52,ReturnSkew,IdioVolAHT,...,DivInit,IndIPO,ExchSwitch,DivOmit,Spinoff,FirmAge,IndMom,betaVIX,VolSD,Mom6m
0,10026.0,197001.0,0.127333,-0.167491,-0.027327,-0.125700,-0.243435,1.033956,-0.460584,-1.638111,...,-0.109765,0.0,0.080845,0.046575,1.648886,0.530691,1.029739,0.254686,0.059106,1.298257
1,10028.0,197001.0,0.216222,-3.470718,-3.564635,-3.283539,-0.450792,0.456415,0.167796,-2.158274,...,-0.109765,0.0,0.080845,0.046575,-0.605813,0.530691,-0.868877,6.433726,0.268360,1.311719
2,10032.0,197001.0,-0.097503,-0.024585,-0.112883,-0.104740,0.486314,-0.044341,0.034855,0.091712,...,-0.109765,0.0,0.080845,0.046575,1.648886,0.530691,0.426197,0.025104,0.046141,0.579390
3,10044.0,197001.0,0.427962,-0.690728,-3.172841,-3.824552,-3.552310,2.240116,-3.248288,-2.771246,...,-0.109765,0.0,0.080845,0.046575,-0.605813,0.530691,1.029739,0.616524,0.192383,0.480053
4,10104.0,197001.0,0.191605,0.181953,-1.257029,-1.072510,-0.301003,1.152558,-0.059413,-0.997067,...,-0.109765,0.0,0.080845,0.046575,-0.605813,0.530691,2.074469,1.918301,-4.854007,-0.813922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
608252,91556.0,202411.0,-0.024872,0.405008,0.523823,0.432284,-0.508538,0.881830,0.728595,0.791906,...,-0.099177,0.0,0.046575,0.087371,0.000000,0.910508,1.430622,0.338326,0.009784,0.436283
608253,91855.0,202411.0,-0.088628,-0.199829,0.029351,0.302339,-0.484865,-0.013330,-0.858826,0.271475,...,-0.099177,0.0,0.046575,0.087371,0.000000,0.884906,0.287300,-0.840123,0.242905,-0.337734
608254,92583.0,202411.0,-0.161971,0.188926,0.066852,-0.085316,0.545978,-0.963746,0.378333,-0.830670,...,-0.099177,0.0,0.046575,0.087371,0.000000,0.997556,-0.313135,0.416040,0.208653,-0.070784
608255,92655.0,202411.0,-0.171552,0.597966,0.581891,0.484267,0.424528,0.444650,1.643249,0.680343,...,-0.099177,0.0,0.046575,0.087371,0.000000,0.859304,0.684406,1.077172,0.100552,0.397531


In [10]:
df_RtnJ.isna().any().any()

np.False_

In [11]:
# 将 RtnI 数据错位一期 因为使用下一期的前沿组合收益进入定价核
df_RtnI['yyyymm'] = df_RtnI['yyyymm'].shift(1)

In [12]:
# 删除最后一个截面的数据
df_RtnI = df_RtnI.dropna(axis=0)
df_RtnJ = df_RtnJ[df_RtnJ['yyyymm'].isin(df_RtnI['yyyymm'])]

In [13]:
df_RtnI

,yyyymm,MaxRet,RealizedVol,IdioVol3F,ReturnSkew3F,High52,ReturnSkew,IdioVolAHT,zerotrade1M,DolVol,...,DivInit,IndIPO,ExchSwitch,DivOmit,Spinoff,FirmAge,IndMom,betaVIX,VolSD,Mom6m
1,197001.0,-0.003664,-0.011480,-0.008748,-0.064380,0.134947,-0.072887,0.015311,-0.015795,-0.017999,...,-0.011912,-0.011691,0.037457,0.008600,-0.010427,0.003701,0.017070,-0.004446,-0.015417,0.113456
2,197002.0,0.029763,-0.002334,-0.005837,-0.074349,0.239440,-0.074835,0.008926,-0.026036,-0.018940,...,-0.060480,-0.059742,0.137510,0.057687,-0.047706,0.117005,0.049380,-0.017822,-0.035985,0.107710
3,197003.0,-0.007777,-0.025206,-0.028268,-0.077262,0.169261,-0.074618,-0.019653,-0.037155,-0.017177,...,-0.032768,-0.031981,0.054877,-0.031981,-0.025512,0.063894,0.057498,-0.003749,-0.044473,0.117711
4,197004.0,0.037362,0.022492,0.008892,-0.059927,0.170072,-0.049957,0.009913,-0.021427,-0.020260,...,-0.028564,-0.028162,-0.050876,-0.011446,-0.021607,0.060272,0.063794,-0.009540,-0.020458,0.148859
5,197005.0,-0.027236,-0.031630,-0.024772,-0.053981,0.051258,-0.060350,0.008169,-0.021769,-0.033819,...,0.021272,0.020575,-0.082092,0.026404,0.016688,-0.061100,-0.043409,0.002535,-0.011842,0.078561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,202406.0,0.039772,0.036900,0.023779,0.017277,0.027705,0.018373,0.035582,0.032851,-0.017830,...,-0.003393,-0.002702,-0.026327,-0.085658,-0.002702,-0.001558,0.013956,0.052306,0.011174,0.013186
655,202407.0,0.032972,0.034728,0.029562,0.014829,0.027237,0.006424,0.041386,0.008148,-0.028561,...,-0.003679,-0.003341,-0.045473,-0.140709,-0.003341,-0.000956,0.000012,0.020603,0.006690,0.003788
656,202408.0,0.029980,0.030067,0.031553,0.008517,0.011594,0.005588,0.015008,0.007575,-0.007354,...,-0.020625,-0.020395,-0.020047,-0.184424,-0.020395,0.015681,0.034512,0.034884,0.007007,0.025307
657,202409.0,0.011822,0.015077,0.025644,0.019869,0.051094,0.003758,0.041436,0.021562,-0.019508,...,0.047765,0.047639,-0.017854,-0.118242,0.047639,0.016440,-0.001255,-0.019756,0.033127,0.028541


In [14]:
df_RtnJ

,permno,yyyymm,RET,MaxRet,RealizedVol,IdioVol3F,ReturnSkew3F,High52,ReturnSkew,IdioVolAHT,...,DivInit,IndIPO,ExchSwitch,DivOmit,Spinoff,FirmAge,IndMom,betaVIX,VolSD,Mom6m
0,10026.0,197001.0,0.127333,-0.167491,-0.027327,-0.125700,-0.243435,1.033956,-0.460584,-1.638111,...,-0.109765,0.0,0.080845,0.046575,1.648886,0.530691,1.029739,0.254686,0.059106,1.298257
1,10028.0,197001.0,0.216222,-3.470718,-3.564635,-3.283539,-0.450792,0.456415,0.167796,-2.158274,...,-0.109765,0.0,0.080845,0.046575,-0.605813,0.530691,-0.868877,6.433726,0.268360,1.311719
2,10032.0,197001.0,-0.097503,-0.024585,-0.112883,-0.104740,0.486314,-0.044341,0.034855,0.091712,...,-0.109765,0.0,0.080845,0.046575,1.648886,0.530691,0.426197,0.025104,0.046141,0.579390
3,10044.0,197001.0,0.427962,-0.690728,-3.172841,-3.824552,-3.552310,2.240116,-3.248288,-2.771246,...,-0.109765,0.0,0.080845,0.046575,-0.605813,0.530691,1.029739,0.616524,0.192383,0.480053
4,10104.0,197001.0,0.191605,0.181953,-1.257029,-1.072510,-0.301003,1.152558,-0.059413,-0.997067,...,-0.109765,0.0,0.080845,0.046575,-0.605813,0.530691,2.074469,1.918301,-4.854007,-0.813922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607329,91556.0,202410.0,0.104531,0.405008,0.523823,0.432284,-0.508538,0.881830,0.728595,0.791906,...,-0.099177,0.0,0.046575,0.087371,0.000000,0.910508,1.430622,0.338326,0.009784,0.436283
607330,91855.0,202410.0,0.050410,-0.199829,0.029351,0.302339,-0.484865,-0.013330,-0.858826,0.271475,...,-0.099177,0.0,0.046575,0.087371,0.000000,0.884906,0.287300,-0.840123,0.242905,-0.337734
607331,92583.0,202410.0,0.123551,0.188926,0.066852,-0.085316,0.545978,-0.963746,0.378333,-0.830670,...,-0.099177,0.0,0.046575,0.087371,0.000000,0.997556,-0.313135,0.416040,0.208653,-0.070784
607332,92655.0,202410.0,0.077057,0.597966,0.581891,0.484267,0.424528,0.444650,1.643249,0.680343,...,-0.099177,0.0,0.046575,0.087371,0.000000,0.859304,0.684406,1.077172,0.100552,0.397531


In [15]:
df_RtnI.to_csv('./cache/ForTrain/df_RtnI.csv', index=False)
df_RtnJ.to_csv('./cache/ForTrain/df_RtnJ.csv', index=False)